In [5]:
import pandas as pd
import hopsworks
import os
import requests

In [6]:
player_stats_gw_url = "https://fantasy.premierleague.com/api/event/{gw}/live/"
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

In [7]:
try:
    with open('./hopsworks/hopsworks-api-key.txt', 'r') as file:
        os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
except:
    print("In production mode")

project = hopsworks.login()
print(project.description)

2025-01-07 15:28:30,875 INFO: Initializing external client
2025-01-07 15:28:30,878 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 15:28:32,335 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1159321
Default project


In [8]:
general_info = requests.get(bootstrap_url).json()
general_stats = pd.DataFrame(general_info["elements"])
general_stats

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,...,130,77,690,309,702,316,650,283,0.00,0.00
1,True,True,100.0,100.0,205651,0,0,-1,1,2,...,29,14,21,4,191,25,38,10,0.90,0.30
2,True,True,100.0,100.0,226597,0,0,4,-4,2,...,51,3,57,16,27,2,9,2,1.05,0.35
3,True,True,75.0,75.0,219847,0,0,-1,1,1,...,13,5,79,14,38,11,42,12,1.03,0.36
4,True,False,0.0,0.0,463748,0,0,0,0,0,...,636,58,459,55,535,61,570,68,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,True,True,75.0,75.0,509291,0,0,0,0,0,...,222,140,185,86,314,142,594,249,1.20,0.18
707,True,True,NaN,NaN,463212,0,0,0,0,0,...,116,70,332,169,473,227,571,231,0.38,0.00
708,True,True,NaN,NaN,517179,0,0,-1,1,0,...,706,236,426,153,396,130,228,87,0.00,0.00
709,True,True,NaN,NaN,596054,0,0,0,0,0,...,464,293,623,264,671,296,627,267,0.00,0.00


In [9]:
fs = project.get_feature_store()

In [10]:
player_fg = fs.get_feature_group("player_features")

2025-01-07 15:28:33,890 WARNING: VersionWarning: No version provided for getting feature group `player_features`, defaulting to `1`.



RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1159321/featurestores/1150024/featuregroups/player_features). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270009,"usrMsg":"feature group name: player_features feature group version: 1","errorMsg":"Featuregroup wasn\'t found."}', error code: 270009, error msg: Featuregroup wasn't found., user msg: feature group name: player_features feature group version: 1

In [4]:
prev_gameweek = player_fg.select(["gameweek"]).read().max()

print(prev_gameweek)

prev_gameweek = next(
    (gw for gw in general_info['events'] if gw['id'] == int(prev_gameweek)), None
)
prev_finished = prev_gameweek['finished']
prev_finished

NameError: name 'player_fg' is not defined

In [85]:
lagged_features = ['minutes', 'goals_scored', 'assists', 'clean_sheets',
                   'goals_conceded', 'own_goals', 'penalties_saved',
                   'penalties_missed', 'yellow_cards', 'red_cards', 'saves',
                   'bonus', 'bps', 'influence', 'creativity', 'threat',
                   'ict_index', 'starts', 'expected_goals', 'expected_assists',
                   'expected_goal_involvements', 'expected_goals_conceded',
                   'in_dreamteam', 'total_points', 'selected', 'transfers_balance',
                   'value', 'was_home', 'opponent_team']

In [86]:
player_details_url = "https://fantasy.premierleague.com/api/element-summary/{element_id}"

In [ ]:
if prev_finished:

    prev_stats = player_stats_gw_url.format(gw=prev_gameweek["id"])
    prev_stats = requests.get(prev_stats).json()
    prev_stats_df = pd.DataFrame(prev_stats["elements"]).drop(columns=["explain", "modified"])
    prev_stats_df = pd.concat(
        [prev_stats_df.drop(columns=["stats"]), prev_stats_df["stats"].apply(pd.Series)],
        axis=1
    )
    prev_stats_df["gameweek"] = prev_gameweek["id"] + 1
    prev_stats_df.rename(columns={"total_points": "points"}, inplace=True)

    for i in range(1, len(prev_stats_df['id'].unique()) + 1):

        player_details_url_formatted = player_details_url.format(element_id=i)
        player_details = requests.get(player_details_url_formatted).json()
        player_details_history_df = pd.DataFrame(player_details["history"])

        total_points = player_details_history_df["total_points"].sum()

        row = player_details_history_df[player_details_history_df['round'] == prev_gameweek["id"]].iloc[0]

        prev_stats_df.loc[prev_stats_df['id'] == i, 'total_points'] = total_points
        prev_stats_df.loc[prev_stats_df['id'] == i, 'selected'] = row['selected']
        prev_stats_df.loc[prev_stats_df['id'] == i, 'transfers_balance'] = row['transfers_balance']
        prev_stats_df.loc[prev_stats_df['id'] == i, 'value'] = row['value']
        prev_stats_df.loc[prev_stats_df['id'] == i, 'was_home'] = row['was_home']
        prev_stats_df.loc[prev_stats_df['id'] == i, 'opponent_team'] = row['opponent_team']


    prev_stats_df.rename(columns={col: f"prev_{col}" for col in prev_stats_df.columns if col not in ["id", "gameweek", "points"]}, inplace=True)

    prev_stats_df = prev_stats_df.dropna(subset=[f'prev_{col}' for col in lagged_features] + ['points'])

    prev_stats_df = prev_stats_df.astype({col: 'float64' for col in prev_stats_df.columns if col not in ["id", "gameweek", "points"]})

    fs = project.get_feature_store()

    player_fg = fs.get_feature_group(
        name="player_features",
        version=1,
    )

    player_fg.insert(prev_stats_df)

    print("Inserted data for gameweek", prev_gameweek["id"] + 1)
    print(prev_stats_df)